In [35]:
## Source Constants
DATA_SOURCE_ROOT = "../../dtc-capstone-data/slack-data" 
COURSE_CHANNEL = "course-data-engineering"
WELCOME_CHANNEL = "welcome"
USERS_DATA = "users.json"
# book-of-the-week
# announcements-course-data-engineering
# shameless-content

In [ ]:
## Target Constants
PROJECT_ID = os.environ.get("GCP_PROJECT_ID", 'dtc-capstone-344019')
BUCKET = os.environ.get("GCP_GCS_BUCKET", 'dtc_data_lake_blissful-scout-339008')

In [4]:
import json
import re

from glob import glob

In [7]:
import pandas as pd

In [63]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('dtc-capstone') \
    .getOrCreate()

## Users

In [106]:
with open(f'{DATA_SOURCE_ROOT}/{USERS_DATA}') as f_in:
    all_users = json.load(f_in)

In [ ]:
# 1. Store Raw Data to datalake...

def upload_to_gcs(bucket, object_name, local_file):
    """
    Ref: https://cloud.google.com/storage/docs/uploading-objects#storage-upload-object-python
    :param bucket: GCS bucket name
    :param object_name: target path & file-name
    :param local_file: source path & file-name
    :return:
    """
    # WORKAROUND to prevent timeout for files > 6 MB on 800 kbps upload speed.
    # (Ref: https://github.com/googleapis/python-storage/issues/74)
    storage.blob._MAX_MULTIPART_SIZE = 5 * 1024 * 1024  # 5 MB
    storage.blob._DEFAULT_CHUNKSIZE = 5 * 1024 * 1024  # 5 MB
    # End of Workaround

    client = storage.Client()
    bucket = client.bucket(bucket)

    blob = bucket.blob(object_name)
    blob.upload_from_filename(local_file)

In [114]:
def format_user_data(d):
    identity = ["name", "real_name"]
    location = ["tz","tz_label", "tz_offset"]
    status = ["deleted", "is_admin", "is_owner", "is_primary_owner",
              "is_restricted","is_ultra_restricted","is_bot", 
              "is_email_confirmed"]
    
    user = {}
    for key, value in {"identifiers": identity, "location":location, "status":status}.items():
        user[key]={}
        user[key]["id"] = d["id"]
        for v in value:
            if v in d:
                user[key][v]=d[v]
            else:
                user[key][v]="None"   
                
    return user

In [115]:
users = [format_user_data(d) for d in all_users]

In [116]:
users_identity = [u["identifiers"] for u in users]
users_location = [u["location"] for u in users]
users_status = [u["status"] for u in users]

In [119]:
schema_id = types.StructType([
    types.StructField('id', types.StringType(), False),
    types.StructField('name', types.StringType(), True),
    types.StructField('real_name', types.StringType(), True)
])

schema_loc = types.StructType([
    types.StructField("tz", types.StringType(), True),
    types.StructField("tz_label", types.StringType(), True),
    types.StructField("tz_offset", types.StringType(), True),
])

schema_status = types.StructType([
    types.StructField("deleted", types.StringType(), True),
    types.StructField("is_admin", types.StringType(), True),
    types.StructField("is_owner", types.StringType(), True),
    types.StructField("is_primary_owner", types.StringType(), True),
    types.StructField("is_restricted", types.StringType(), True),
    types.StructField("is_ultra_restricted", types.StringType(), True),
    types.StructField("is_bot", types.StringType(), True),
    types.StructField("is_email_confirmed", types.StringType(), True),
])

# Create data frame
df_users_identity = spark.createDataFrame(users_identity, schema_id)
df_users_location = spark.createDataFrame(users_location, schema_loc)
df_users_status = spark.createDataFrame(users_status, schema_status)

In [ ]:
# Store Raw data..
...

In [ ]:
# Partition Data and read to dwh

In [ ]:
# Partition Data
df.repartition(24) \
    .write.parquet('data/pq/fhvhv/', mode="overwrite")

In [85]:
ordersDf = spark.read.format("json")
                .option("inferSchema", "true")
                .option("multiLine", "true")
                .load(users_identity)

IndentationError: unexpected indent (3219257195.py, line 2)

In [28]:
files_course_channel = sorted(glob(f'{DATA_SOURCE_ROOT}/{COURSE_CHANNEL}/*.json'))
len(files_course_channel)

136

In [30]:
## yyyy-mm-dd
start, end = files_course_channel[0].split("/")[-1].split(".")[0], files_course_channel[-1].split("/")[-1].split(".")[0]
(start,end)

('2020-11-22', '2022-03-14')

In [31]:
def load_docs(files):
    all_docs = []

    for f in files:
        with open(f) as f_in:
            docs = json.load(f_in)
            all_docs.extend(docs)
    
    return all_docs

In [32]:
def clean_user(d):
    p = d['profile']
    name = p['display_name']
    if len(name) == 0:
        name = p['real_name']
    return {
        'name': name,
        'image': p['image_72']
    }